# 0. Installing Transformers and Importing Dependencies

In [1]:
!pip install transformers

  Using cached tokenizers-0.13.3-cp311-cp311-win_amd64.whl (3.5 MB)
                                              0.0/277.5 kB ? eta -:--:--
     -                                        10.2/277.5 kB ? eta -:--:--
     ----                                  30.7/277.5 kB 435.7 kB/s eta 0:00:01
     ------------                          92.2/277.5 kB 751.6 kB/s eta 0:00:01
     ---------------------                  153.6/277.5 kB 1.0 MB/s eta 0:00:01
     ------------------------------         225.3/277.5 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 277.5/277.5 kB 1.2 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 1. Load Summarization Pipeline

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# 2. Get Blog Post from Medium

In [4]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [5]:
r = requests.get(URL)

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [7]:
ARTICLE

'Will The Game Stop with Gamestop Or Is This Just The Beginning? Crypto, Markets, Trading Receive Stories from @TK'

# 3. Chunk Text

In [8]:
max_chunk = 500

In [9]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [10]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [11]:
len(chunks)

1

# 4. Summarize Text

In [12]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

Your max_length is set to 120, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


In [13]:
res[0]

{'summary_text': 'Crypto, Markets, Trading Receive Stories from @TK or is This Just The Beginning? @TK, markets, trading, crypto, markets, trading .'}

In [14]:
' '.join([summ['summary_text'] for summ in res])

'Crypto, Markets, Trading Receive Stories from @TK or is This Just The Beginning? @TK, markets, trading, crypto, markets, trading .'

In [15]:
text = ' '.join([summ['summary_text'] for summ in res])

# 5. Output to Text File

In [16]:
text

'Crypto, Markets, Trading Receive Stories from @TK or is This Just The Beginning? @TK, markets, trading, crypto, markets, trading .'

In [17]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)